<a href="https://colab.research.google.com/github/vanecornejo/EDP-II/blob/main/M%C3%A9todo%20de%20Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Método de Jacobi**

El método de Jacobi es una técnica iterativa que sirve para encontrar soluciones aproximadas a sistemas de ecuaciones lineales del tipo $Ax=b$, despejando cada incógnita de su respectiva ecuación y generando una secuencia de aproximaciones hasta alcanzar la precisión deseada.

Este programa implementa el método de Jacobi para resolver el sistema
de ecuaciones lineales resultante de hacer la ecuación de Laplace
con condiciones de frontera


Implementando el código al primer ejercicio del problemario:

Queremos aproximar la solución de la ecuación de Laplace en la región
$$ 0 \leq x \leq 2, \quad 0 \leq y \leq 2 $$
con condiciones de frontera dadas:

- En los bordes verticales:
$$ u(0,y) = 0, \quad u(2,y) = y(2-y), \quad 0 < y < 2 $$

- En los bordes horizontales:
$$
u(x,0) = 0, \quad
u(x,2) =
\begin{cases}
x, & 0 < x < 1 \\
2-x, & 1 \leq x < 2
\end{cases}
$$

Usamos una malla de tamaño $ h = \frac{2}{3} $.  
Esto significa que los puntos interiores están en:
$$
x = \tfrac{2}{3}, \tfrac{4}{3}, \quad
y = \tfrac{2}{3}, \tfrac{4}{3}.
$$

Es decir, tenemos

$$
u_1 = u\!\left(\tfrac{2}{3}, \tfrac{2}{3}\right), \quad
u_2 = u\!\left(\tfrac{4}{3}, \tfrac{2}{3}\right), \quad
u_3 = u\!\left(\tfrac{2}{3}, \tfrac{4}{3}\right), \quad
u_4 = u\!\left(\tfrac{4}{3}, \tfrac{4}{3}\right).
$$

## Sistema de ecuaciones
De la fórmula de Laplace:

$$ u_{i,j} = \frac{1}{4}\big(u_{i+1,j} + u_{i-1,j} + u_{i,j+1} + u_{i,j-1}\big) $$

y aplicando las condiciones de frontera, obtenemos:

$$
\begin{aligned}
u_1 &= \tfrac{1}{4}(u_2 + u_3), \\[6pt]
u_2 &= \tfrac{1}{4}(u_1 + u_4 + \tfrac{8}{9}), \\[6pt]
u_3 &= \tfrac{1}{4}(u_1 + u_4 + \tfrac{2}{3}), \\[6pt]
u_4 &= \tfrac{1}{4}(u_2 + u_3 + \tfrac{14}{9}).
\end{aligned}
$$

In [43]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd

Vamos a definir una función llamada jacobi, donde vamos a tener como entradas:
*   La matriz $A$ (nxn)
*   El vector $b$ (n)
*   La aproximación lineal $x0$
*   La tolerancia $Tol$
*   Número máximo de iteraciones $N$

Y nuestras salidas serán:

*   (x, tabla, exito)
*   x -> Vector con la solución aproximada $x$
*   tabla -> DataFrame con las iteraciones, variables y error
*   exito -> True si converge, False si alcanza N iteraciones sin converger

In [44]:
def jacobi(A, b, x0, tol, N):
    n = len(A)

    # Guardamos el vector inicial
    XO = np.array(x0, dtype=float)

    # Creamos una lista donde vamos a guardar los resultados de cada iteración
    resultados = []

    k = 1 # Contador de iteraciones
    while k <= N:
        x1 = np.zeros_like(XO)  # Para la nueva aproximación

        # Calculamos la nueva aproximación
        for i in range(n):

            # Calculamos la suma de a_ij * x_j para j distinto de i
            suma = sum(A[i][j]*XO[j] for j in range(n) if j != i)

            # Fórmula de Jacobi: x_i = (b_i - suma)/a_ii
            x1[i] = (b[i] - suma)/A[i][i]

        # Calculamos el error ||x - XO||
        error = np.linalg.norm(x1 - XO, ord=np.inf)

        # Guardamos los resultados de esta iteración en la lista creada anteriormente
        resultados.append([k] + list(x1) + [error])

        if error < tol:
            # Creamos un DataFrame con los resultados en forma de tabla
            df = pd.DataFrame(resultados, columns=["Iteración"] + [f"u{i+1}" for i in range(n)] + ["Error"])
            return x1, df, True

        # Actualizamos XO para la siguiente iteración
        XO = x1
        k += 1

    # Si llegamos aquí, significa que no convergió en N iteraciones
    df = pd.DataFrame(resultados, columns=["Iteración"] + [f"u{i+1}" for i in range(n)] + ["Error"])
    return XO, df, False

Ponemos los datos obtenidos del ejercicio para verificar.

In [45]:
A = np.array([
    [1, -1/4, -1/4, 0],    # Ecuación de u1
    [-1/4, 1, 0, -1/4],    # Ecuación de u2
    [-1/4, 0, 1, -1/4],    # Ecuación de u3
    [0, -1/4, -1/4, 1]     # Ecuación de u4
], dtype=float)


b = np.array([
    0,        # Constante en ecuación de u1
    2/9,      # Constante en ecuación de u2
    1/6,      # Constante en ecuación de u3
    7/18      # Constante en ecuación de u4
], dtype=float)

x0 = np.zeros(4) # Aproximación inicial
tol = 1e-6   # Tolerancia
N = 100      # Número máximo de iteraciones

sol, tabla, exito = jacobi(A, b, x0, tol, N)

print("Resultados:")
print(tabla)

if exito:
    print("\nConvergió a la solución aproximada en", len(tabla), "iteraciones.")
else:
    print("\nMáximo de iteraciones alcanzado sin converger.")

print("Valores aproximados de u:", sol)


Resultados:
    Iteración        u1        u2        u3        u4         Error
0           1  0.000000  0.222222  0.166667  0.388889  3.888889e-01
1           2  0.097222  0.319444  0.263889  0.486111  9.722222e-02
2           3  0.145833  0.368056  0.312500  0.534722  4.861111e-02
3           4  0.170139  0.392361  0.336806  0.559028  2.430556e-02
4           5  0.182292  0.404514  0.348958  0.571181  1.215278e-02
5           6  0.188368  0.410590  0.355035  0.577257  6.076389e-03
6           7  0.191406  0.413628  0.358073  0.580295  3.038194e-03
7           8  0.192925  0.415148  0.359592  0.581814  1.519097e-03
8           9  0.193685  0.415907  0.360352  0.582574  7.595486e-04
9          10  0.194065  0.416287  0.360731  0.582954  3.797743e-04
10         11  0.194255  0.416477  0.360921  0.583143  1.898872e-04
11         12  0.194350  0.416572  0.361016  0.583238  9.494358e-05
12         13  0.194397  0.416619  0.361064  0.583286  4.747179e-05
13         14  0.194421  0.416643  0